In [2]:
print("sub dude")

sub dude


In [3]:
%pwd

'c:\\Users\\ACER\\Desktop\\LLM\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\ACER\\Desktop\\LLM'

In [8]:
pip install langchain


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   - -------------------------------------- 0.8/15.8 MB 3.7 MB/s eta 0:00:05
   --- ------------------------------------ 1.6/15.8 MB 3.8 MB/s eta 0:00:04
   ----- ---------------------------------- 2.4/15.8 MB 3.8 MB/s eta 0:00:04
   ------- -------------------------------- 3.1/15.8 MB 3.7 MB/s eta 0:00:04
   --------- ------------------------------ 3.9/15.8 MB 3.7 MB/s eta 0:00:04
   ----------- ---------------------------- 4.7/15.8 MB 3.7 MB/s eta 0:00:04
   ------------- -------------------------- 5.5/15.8 MB 3.6 MB/s eta 0:00:03
   --------------- ------------------------ 6.0/15.8 MB 3.7 MB/s eta 0:00:03
   ----------------- ---------------------- 6.8/15.8 MB 3.6 MB/s eta 0:00:03
   ------------------- -------------------- 7.6/15.8 MB 3.6 MB/s eta 0:00:03
   ------------

In [10]:
!pip install -U langchain-community


  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 6.6 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.2 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='DATA/')

In [9]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 42275


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\ACER\AppData\Local\Temp\ipykernel_48676\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

api_key = os.getenv("PINECONE_API_KEY")


In [23]:
index_name = "medicalchatbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [24]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")



In [25]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [26]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [29]:
retrieved_docs = retriever.invoke("What is Fever?")


In [30]:
retrieved_docs

[Document(id='ec3fd1a2-56fa-45a7-8f48-c87e9b74c433', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-01-26T18:46:42+00:00', 'page': 1545.0, 'page_label': '1546', 'producer': 'iLovePDF', 'source': 'DATA\\Medical inforamtaion data.pdf', 'total_pages': 4970.0}, page_content='Description\nFever is a natural response of the body that helps\nin fighting off foreign substances, such as microorgan-\nisms, toxins, etc. Body temperature is set by the\nthermoregulatory center, located in an area in the\nbrain called hypothalamus. Body temperature is not\nconstant all day, but actually is lowest at 6 A.M. and\nhighest around 4–6 P.M. In addition, temperature\nvaries in different regions of the body; for example,\nrectal and urine temperatures are about one degree'),
 Document(id='1f7ec766-d640-4922-b1d1-ee797fc7f146', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-01-26T18:46:42+00:00', 'page': 1846.0, 'page_label': '1847', 'producer': 'iLovePDF', 'source': '

In [31]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "what is fever?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}